# Ciencia de datos
## Práctica 4. Visualización de la información con plotly
### Alberto Benavides

De los datos correspondientes a los registros obtenidos de los PDFs de la Secretaría de Salud de México

In [110]:
import pandas as pd

data1 = pd.read_csv("D:/FIME/Epidemia/Data/csvSemanales/nacional.csv")
data1.loc[:, 'cie'] = data1['cie'].astype(str).str[0]

print(data1.sample(3))

       anio  sem                                     enfermedad  casos cie
15988  2013   21                                        tetanos      4   a
16961  2013   45  infecciones intestinales por otros organismos  79202   a
22822  2014   46                                   oncocercosis      0   b


y estos mismos preprocesados

In [111]:
data2 = pd.read_csv("D:/FIME/Epidemia/Data/semanalesTodasKmeans.csv")
print(data2.sample(3))

              # m        f1        f2        f3        f4       ac1       ac2  \
102 -3.415412e-05  0.020000  0.022000  0.018000  0.004000  0.942775  0.885585   
77   1.682277e-10  0.002083  0.025000  0.027083  0.022917 -0.002164 -0.002169   
20   5.784277e-05  0.018519  0.037037  0.074074  0.277778  0.762638  0.618995   

          ac4       ac8      ac16      ac32                          enf cie  \
102  0.775846  0.480766 -0.045256 -0.076870  NEUMONIAS Y BRONCONEUMONIAS   J   
77  -0.002178 -0.002197 -0.002234 -0.000149             TETANOS NEONATAL   A   
20   0.439318  0.062748 -0.230947 -0.133923                CONJUNTIVITIS   H   

     cluster  
102        3  
77         4  
20         5  


se pueden mostrar visualizaciones de los resultados de las estadísticas básicas reportadas. Empezaremos por mostrar un diagrama de cantidad de registros por letra inicia de CIE de los datos extraidos de los PDFs

In [112]:
 # https://plot.ly/python/pie-charts/
print(data1['cie'].value_counts())

a    10017
b     5010
t     1460
j     1242
u      999
w      681
i      665
g      548
x      454
p      440
z      406
e      364
n      208
q      208
f      207
k      181
y      111
h      104
c      104
v      104
s      104
r       52
o       52
Name: cie, dtype: int64


De modo que las enfermedades cuya CIE inicia con la letra A ocupan el 42.2% de los registros. 

Ahora bien, se puede obtener una descripción de los datos agrupados por número de casos registrados y letra inicial de CIE

In [113]:
# https://plot.ly/python/box-plots/
import numpy as np
boxes = []
plt.figure()
for cie in data1.groupby(['cie']):
    boxes.append(np.log(cie[1][cie[1]['casos'] > 0]['casos']))
plt.boxplot(boxes)
plt.savefig('P4/boxplotCIE')
plt.show()
plt.close()

Al preprocesar los datos, se agrupan las enfermedades por primera letra de la CIE y se descubre que los grupos A y B contienen la mayoría de los registros, contando un 31.19% y un 19.57% respectivamente.

In [114]:
# https://stackoverflow.com/a/51453257
pd.options.display.max_columns

print(data2['cie'].value_counts())

A    41
B    22
T     8
E     7
J     6
I     5
K     4
F     4
N     4
Q     4
U     3
G     3
Y     2
S     2
C     2
V     2
H     2
W     2
P     1
O     1
Z     1
X     1
R     1
Name: cie, dtype: int64


In [115]:
import matplotlib as mpl 
mpl.use('agg')
import matplotlib.pyplot as plt

for cie in data2.groupby(['cluster']):
    fig = plt.figure(figsize=(22,10))
    plt.title('Cluster {}'.format(cie[0]))
    i = 1
    for feature in cie[1]:
        if feature == "enf":
            break
        else:
            # https://stackoverflow.com/a/28396902
            ax = fig.add_subplot(1, 11, i)
            ax.set_xlabel(feature)
            i = i + 1
            # ax.boxplot(np.array(cie[1][feature]).astype(np.float))
            plt.boxplot(cie[1][feature])
    fig.savefig('P4/cluster{}.png'.format(cie[0]))
    fig.tight_layout(pad=0.4, w_pad=1, h_pad=1.0)
    plt.close(fig)

In [116]:
for feature in data2:
    if feature != "enf" and feature != 'cie' and feature != 'cluster':
        fig = plt.figure(figsize=(20,10))
        plt.title(feature)
        boxes = []
        i = 1
        for clust in data2.groupby(['cluster']):
            # https://stackoverflow.com/a/28396902
            i = i + 1
            boxes.append(clust[1][feature])
        plt.boxplot(boxes)
        fig.savefig('P4/{}.png'.format(feature))
        fig.tight_layout(pad=0.4, w_pad=1, h_pad=1.0)
        plt.close(fig)

In [117]:
from sklearn.decomposition import PCA
import matplotlib.cm as cm
pca = PCA(n_components=3)

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
ti = data2.loc[:, data2.columns != 'enf']

ti = ti.loc[:, ti.columns != 'cie']
ti = ti.loc[:, ti.columns != 'cluster']
features = pca.fit_transform(ti)

plt.close(fig)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features[:,0], features[:,1], features[:,2], c=data2['cluster'], cmap=cm.Set1)
plt.savefig("P4/scater3d.png")
plt.show()
plt.close(fig)

In [118]:
pca = PCA(n_components=2)

features = pca.fit_transform(ti)

fig = plt.figure()
plt.scatter(features[:,0], features[:,1], c=data2['cluster'], cmap=cm.Set1)
plt.savefig("P4/scater.png")
plt.close(fig)

También se obtuvieron las [correlaciones](https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/) existentes entre las características de los datos preprocesados, siendo de interés aquéllas que guardan correlación con el tipo de cluster asignado por $k$-medias

In [119]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html
data2['cie'], uniques = pd.factorize(data2['cie']) 

#https://stackoverflow.com/a/19483025
#print(list(data.corr()))

import seaborn as sns

fig = plt.figure() 
corr = data2.corr()
plt.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns, rotation='vertical');
plt.yticks(range(len(corr.columns)), corr.columns);
# https://stackoverflow.com/a/42323184
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
plt.savefig("P4/corr.png")
plt.close(fig)

Finalmente, se seleccionan las características de los datos

In [120]:
features = ["m", "f1", "f2", "f3", "f4", "ac1", "ac2", "ac4", "ac8", "ac16", "ac32"]
x = data2.loc[:, features].values

se normalizan 

In [121]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
from sklearn.preprocessing import MinMaxScaler
x = MinMaxScaler().fit(x).transform(x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

y con estas características normalizadas se puede hacer una selección a partir del umbral de varianza

In [ ]:
# https://stackoverflow.com/a/7670325
print("Columnas iniciales = {}".format(x.shape[1]))

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold
from sklearn.feature_selection import VarianceThreshold
th = 0.01 # .8 * (1 - .8)
print("Umbral de varianza = {}".format(th))
sel = VarianceThreshold(threshold=th)
x = sel.fit_transform(x)
# https://stackoverflow.com/q/39812885
dataSelected = data2[data2.columns[sel.get_support(indices=True)]]

y mostrar sus correlaciones

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
dataSelected = dataSelected.assign(cluster=data2[['cie']])

fig = plt.figure()
corr = dataSelected.corr()
plt.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns, rotation='vertical');
plt.yticks(range(len(corr.columns)), corr.columns);
# https://stackoverflow.com/a/42323184
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
plt.savefig("P4/corrSelected.png")
plt.show()
plt.close(fig)

In [ ]:
trace = go.Scatter(
    x = data['ac2'],
    y = data['f1'],
    mode = 'markers',
     marker=dict(
        color=labels.astype(data['cluster'])
    )
)

scatter = [trace]

py.iplot(scatter, filename='basic-scatter')

* Boxplots de features contra clusters; otro de clusters contra features
* Gráfica por cluster qué series de tiempo tiene cada cluster
* Incluir lo de síntomas (de preferencia en inglés)
* Grupos parecidos; puede haber un grupo... Explicar grupos (definir) 
* Revisar clusters significativos
* Utilizar sólo cies reportadas en todos los años y revisar si estadísticamente son distintas


P6

* Cluster id por atributos
* Cluster contra otros clusters
* Cluster por letras

Regresión múltiple
* Cie y cluster en función de los atributos